In [108]:
import pandas as pd
import numpy as np
from pulp import *

In [109]:
nr_pos = np.random.randint(low=4, high=5)
choices_per_pos = np.random.randint(low=5, high=7, size=nr_pos)

In [110]:
muls=list()
dims=list()
for mul in range(0,nr_pos):
    for dim in range(0,choices_per_pos[mul]):
        muls.append('mul_'+str(mul))
        dims.append('dim_'+str(dim))
df = pd.DataFrame(data = {'pos': muls,
                         'choices': dims}
                  , columns=['pos','choices'])
df.loc[:, 'value'] = np.random.randint(low=2, high=8, size=len(df))

In [111]:
pos_choice = pulp.LpVariable.dicts("pos_choice",
                                     ([tuple(x) for x in df[['pos','choices']].values]),
                                     cat='Binary')

In [112]:
model = pulp.LpProblem("dummy", pulp.LpMaximize)

model += pulp.lpSum(
    [pos_choice[x[0], x[1]] * float(df[(df.pos == x[0]) & (df.choices == x[1])]['value']) for x in df[['pos','choices']].values]
)

In [113]:
for mull in df.pos.unique():
    model += sum([pos_choice[x[0], x[1]] for x in df[['pos','choices']].values if x[0]==mull]) <= 1
    model += sum([pos_choice[x[0], x[1]] for x in df[['pos','choices']].values if x[0]==mull]) >= 1

model += pulp.lpSum(
    [pos_choice[x[0], x[1]] * float(df[(df.pos == x[0]) & (df.choices == x[1])]['value']) for x in df[['pos','choices']].values]
)<=13

In [114]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [115]:
output = []
for pos, choice in df[['pos','choices']].values:
    if pos_choice[(pos, choice)].varValue==1:
        var_output = {
            'pos': pos,
            'choices': choice,
            'choice_made': pos_choice[(pos, choice)].varValue
        }
        output.append(var_output)
output_df = pd.DataFrame.from_records(output)

In [116]:
output_df

,choice_made,choices,pos
0,1.0,dim_4,mul_0
1,1.0,dim_4,mul_1
2,1.0,dim_4,mul_2
3,1.0,dim_4,mul_3


In [117]:
df.merge(output_df)

,pos,choices,value,choice_made
0,mul_0,dim_4,3,1.0
1,mul_1,dim_4,2,1.0
2,mul_2,dim_4,3,1.0
3,mul_3,dim_4,5,1.0
